# Concept Drift Modelling

In [1]:
import os
import arrow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandapower as pp

plt.style.use('tableau-colorblind10')

/Users/torge/miniconda3/envs/ma_modells/lib/python3.7/site-packages/pandapower/io_utils.py:8: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_series_equal, assert_frame_equal


In [2]:
from GridExecutor import Executor
from ConceptDrift import Drifter

## Set configs

In [3]:
cwd = os.getcwd()
print(cwd)
data_fn = os.path.join(cwd, 'data_modelling')
grid_fn = os.path.join(cwd, 'grid_modelling')
fig_fn = os.path.join(cwd, 'figs')
print(data_fn)
print(grid_fn)

/Users/torge/Development/master/masterthesis_code/02_Experimente/DataGenerator
/Users/torge/Development/master/masterthesis_code/02_Experimente/DataGenerator/data_modelling
/Users/torge/Development/master/masterthesis_code/02_Experimente/DataGenerator/grid_modelling


In [4]:
pp.__version__

'2.2.1'

## Load Net from JSON

In [5]:
fn = os.path.join(grid_fn, 'CIGRE_net.json')
grid = pp.from_json(fn)

## Load Data

In [6]:
fn = os.path.join(data_fn, 'main_agg.h5')
all_data = pd.read_hdf(fn, key='df')
fn2 = os.path.join(data_fn, 'main.csv')
unscaled_data = pd.read_csv(fn2, sep=';', index_col='index')

In [7]:
start_date = '2023-01-01 00:00:00'
arrw_start = arrow.get(start_date)
end_date = '2023-12-31 23:45:00'
arrw_end = arrow.get(end_date)
res = arrw_end - arrw_start
duration_days = res.days + 1
print(duration_days)

365


In [8]:
data = all_data.loc[start_date:end_date].copy()
data_unscaled = unscaled_data.loc[start_date:end_date].copy()

### Load Load Mapping

In [9]:
fn = os.path.join(data_fn, 'load_mapping.npy')
init_load_mapping =  np.load(fn,allow_pickle='TRUE').item()

### Scale some columns since they are in kW and not MW

In [10]:
drop_cols = data_unscaled.columns
data_unscaled['load_h0_normed_MW'] = data_unscaled['load_h0_normed_kW'] / 1000
data_unscaled['load_g0_normed_MW'] = data_unscaled['load_g0_normed_kW'] / 1000
data_unscaled['load_l0_normed_MW'] = data_unscaled['load_l0_normed_kW'] / 1000
data_unscaled['gen_pv_normed_MW'] = data_unscaled['gen_pv_normed_kW'] 
data_unscaled['gen_wind_normed_MW'] = data_unscaled['gen_wind_normed_kW']
data_unscaled['gen_gas_normed_MW'] = data_unscaled['gen_gas_normed_kW'] 

data_unscaled.drop(drop_cols, axis=1, inplace=True)

In [11]:
timestamps = [x[0] for x in data.index]
timestamps = list(dict.fromkeys(timestamps))

In [12]:
grid.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type,controllable
0,AGG_BUS_1,1,0.196966,0.095395,0.0,0.0,NaN,1.0,True,None,False
1,AGG_BUS_2,2,0.382200,0.185108,0.0,0.0,NaN,1.0,True,None,False
2,AGG_BUS_3,3,0.063200,0.030609,0.0,0.0,NaN,1.0,True,None,False
3,AGG_BUS_4,4,0.065400,0.031675,0.0,0.0,NaN,1.0,True,None,False
4,AGG_BUS_5,5,0.282300,0.136724,0.0,0.0,NaN,1.0,True,None,False
5,AGG_BUS_6,6,0.159500,0.077249,0.0,0.0,NaN,1.0,True,None,False
6,AGG_BUS_7,7,0.224900,0.108924,0.0,0.0,NaN,1.0,True,None,False
7,AGG_BUS_8,8,0.094100,0.045575,0.0,0.0,NaN,1.0,True,None,False
8,AGG_BUS_9,9,0.282300,0.136724,0.0,0.0,NaN,1.0,True,None,False
9,AGG_BUS_10,10,0.000000,0.000000,0.0,0.0,NaN,1.0,True,None,False


## Define Conept Drifs

In [13]:
manipulate_switch =  [
    {
        'switch_id': 4,
        'set_closed': True,
        'at_time_idx': 20,
        'until_time_idx': 80
    },
    {
        'switch_id': 4,
        'set_closed': False,
        'at_time_idx': 80
    },
    {
        'switch_id':2,
        'set_closed': True,
        'at_time_idx': 96 * 62,
        'until_time_idx': 96 * 96
    },
    {
        'switch_id':2,
        'set_closed': False,
        'at_time_idx': 96 * 96
    },
]

load_mapping = [
    {
        'bus_id': 1,
        'load_mapping': [3, 0, 2, 0, 1, 0],
        'at_time_idx': 96 * 24,
        'until_time_idx': 96 * 64
    },
    {
        'bus_id': 3, # three means -> 'AGG_BUS_3'
        'load_mapping': [2, 1, 0, 0, 1, 0],
        'at_time_idx': 96 * 100,
        'until_time_idx': -1
    },
    {
        'bus_id': 10, 
        'load_mapping': [2, 1, 1, 0, 1, 0],
        'at_time_idx': 31 * 97,
        'until_time_idx': 36 * 96 
    },
    {
        'bus_id': 11,
        'load_mapping': [2, 1, 0, 0, 2, 0],
        'at_time_idx': 245 * 97,
        'until_time_idx': -1 
    },
    
]

change_cos_phi = [
    {
        'bus_id': 1,
        'load_mapping': [3, 0, 2, 0, 1, 0], # if None, use initial load_mapping!
        'load_id_in_load_mapping': [0],
        'cos_phi': 0.85,
        'at_time_idx': 323,
        'until_time_idx': 432
    },
    {
        'bus_id': 4,
        'load_mapping': None, # if None, use initial load_mapping!
        'load_id_in_load_mapping': [0],
        'cos_phi': 0.89,
        'at_time_idx': 400,
        'until_time_idx': 400 + 96 * 14
    },
    {
        'bus_id': 13,
        'load_mapping': None, # if None, use initial load_mapping!
        'load_id_in_load_mapping': [0],
        'cos_phi': 0.94,
        'at_time_idx': 96 * 100,
        'until_time_idx': 96 * 131
    },
    {
        'bus_id': 8,
        'load_mapping': [2, 0, 4, 0, 1, 0], # if None, use initial load_mapping!
        'load_id_in_load_mapping': [0,2],
        'cos_phi': 0.94,
        'at_time_idx': 96 * 200,
        'until_time_idx': 96 * 250
    },
    {
        'bus_id': 14,
        'load_mapping': [1, 0, 2, 0, 1, 0], # if None, use initial load_mapping!
        'load_id_in_load_mapping': [0,4],
        'cos_phi': 0.96,
        'at_time_idx': 96 * 180,
        'until_time_idx': 96 * 220
    },
    {
        'bus_id': 6,
        'load_mapping': None, # if None, use initial load_mapping!
        'load_id_in_load_mapping': [0,1],
        'cos_phi': 0.84,
        'at_time_idx': 96 * 23,
        'until_time_idx': 96 * 42
    },
    
]

concept_drift = {'manipulate_switch': manipulate_switch, 'load_mapping': load_mapping, 'change_cos_phi': change_cos_phi}

In [14]:
duration = duration_days * 96
print('Duration: {} 1/4 hours'.format(duration))

Duration: 35040 1/4 hours


In [15]:
drifter = Drifter(grid=grid, concept_drifts=concept_drift, load_mapping=init_load_mapping, unscaled_data=data_unscaled, timestamp_list=timestamps, agg_data=data.copy())
executor = Executor(grid)

In [16]:
labels = drifter.annotate_drifting_labels()

## Manipulate Load Mapping before simulation

In [17]:
drifted_data = drifter.manipulate_load_mapping()

++++++++++++++++++++++++++++++
On 2023-01-25 00:00:00 change load mapping at bus: 1 until 2023-03-06 00:00:00
Old load mapping was: [2 1 0 0 0 1]
New load mapping is: [3, 0, 2, 0, 1, 0]
Manipulate 3841 samples!
++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++
On 2023-04-11 00:00:00 change load mapping at bus: 3 until 2023-12-31 23:45:00
Old load mapping was: [2 0 0 0 1 0]
New load mapping is: [2, 1, 0, 0, 1, 0]
Manipulate 25440 samples!
++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++
On 2023-02-01 07:45:00 change load mapping at bus: 10 until 2023-02-06 00:00:00
Old load mapping was: [0 0 0 0 0 0]
New load mapping is: [2, 1, 1, 0, 1, 0]
Manipulate 450 samples!
++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++
On 2023-09-05 13:15:00 change load mapping at bus: 11 until 2023-12-31 23:45:00
Old load mapping was: [2 0 0 0 0 0]
New load mapping is: [2, 1, 0, 0, 2, 0]
Manipulate 11275 samples!
++++++++++++++++++++++++++++++


In [18]:
drifted_data = drifter.manipulate_cos_phi()

++++++++++++++++++++++++++++++
From 2023-01-04 08:45:00 (idx: 323) change cos phi from load [0] at bus 1 with cos_phi of 0.85 until 2023-01-05 12:00:00 (idx: 433)
Use load mapping: [3, 0, 2, 0, 1, 0]
Manipulate 110 samples!
++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++
From 2023-01-05 04:00:00 (idx: 400) change cos phi from load [0] at bus 4 with cos_phi of 0.89 until 2023-01-19 04:00:00 (idx: 1745)
Use load mapping: [0 1 0 0 0 0]
Manipulate 1345 samples!
++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++
From 2023-04-11 00:00:00 (idx: 9600) change cos phi from load [0] at bus 13 with cos_phi of 0.94 until 2023-05-12 00:00:00 (idx: 12577)
Use load mapping: [2 0 2 0 2 0]
Manipulate 2977 samples!
++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++
From 2023-07-20 00:00:00 (idx: 19200) change cos phi from load [0, 2] at bus 8 with cos_phi of 0.94 until 2023-09-08 00:00:00 (idx: 24001)
Use load mapping: [2, 0, 4, 0, 1, 0]
Manipulate 4801 samples!
++++++

## Execute Simulation

In [19]:
def do_concept_drift(t, grid):
    events_manipulate_switch = drifter.check_timestamps(t)

    if events_manipulate_switch[0]:
        for thing1 in events_manipulate_switch[1]:                
            grid = drifter.manipulate_switch(thing1)

    return grid

In [20]:
for t in range(duration):
    # run simulation
    try:
        grid = do_concept_drift(t, grid)
        
        executor.update_grid(grid)
        ts = timestamps[t]
        frame = data.loc[ts]
        grid = executor.step(frame)
        
        if t%10000 == 0:
            print('Curent step: {}'.format(t))

    except Exception as e:
        print('Exception in timestep: {}!'.format(t))
        print(e)

drifter.simulation_done = True

Curent step: 0
++++++++++++++++++++++++++++++
In step 20 do:
Manipulate Switch #: 4
Set Switch to Status: True
++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++
In step 80 do:
Manipulate Switch #: 4
Set Switch to Status: False
++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++
In step 5952 do:
Manipulate Switch #: 2
Set Switch to Status: True
++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++
In step 9216 do:
Manipulate Switch #: 2
Set Switch to Status: False
++++++++++++++++++++++++++++++
Curent step: 10000
Curent step: 20000
Curent step: 30000


In [21]:
simulation_data = executor.get_simulation_data()

In [22]:
simulation_data['drift_labels'] = labels

In [23]:
assert len(timestamps) == len(simulation_data)

In [24]:
simulation_data.index = timestamps

In [25]:
simulation_data.iloc[10:25]

,line_1_1_loading,line_2_3_loading,line_3_4_loading,line_4_5_loading,line_5_6_loading,line_7_8_loading,line_8_9_loading,line_9_10_loading,line_10_11_loading,line_3_8_loading,...,bus_14_q_mvar,switch_0_status,switch_1_status,switch_2_status,switch_3_status,switch_4_status,switch_5_status,switch_6_status,switch_7_status,drift_labels
2023-01-01 02:30:00,13.888975,11.928320,5.448811,4.546562,1.896291,2.914796,4.550686,1.798720,1.839465,8.148957,...,-0.027025,True,False,False,True,False,True,True,True,0.0
2023-01-01 02:45:00,13.450723,11.588971,5.314201,4.439872,1.848058,2.837975,4.452277,1.759557,1.799972,7.955450,...,-0.027267,True,False,False,True,False,True,True,True,0.0
2023-01-01 03:00:00,13.063863,11.284515,5.197065,4.342048,1.808575,2.778633,4.354152,1.720517,1.760586,7.779621,...,-0.027413,True,False,False,True,False,True,True,True,0.0
2023-01-01 03:15:00,12.686926,10.993506,5.097205,4.252942,1.777781,2.736677,4.256182,1.681550,1.721256,7.621192,...,-0.027994,True,False,False,True,False,True,True,True,0.0
2023-01-01 03:30:00,12.411827,10.768861,5.014910,4.177082,1.753615,2.705761,4.169360,1.647027,1.686396,7.487013,...,-0.027945,True,False,False,True,False,True,True,True,0.0
2023-01-01 03:45:00,12.184635,10.580587,4.945676,4.112135,1.733845,2.681450,4.093491,1.616868,1.655928,7.372453,...,-0.027849,True,False,False,True,False,True,True,True,0.0
2023-01-01 04:00:00,11.970337,10.401079,4.880805,4.049413,1.716259,2.661499,4.017719,1.586756,1.625496,7.262328,...,-0.027752,True,False,False,True,False,True,True,True,0.0
2023-01-01 04:15:00,11.825755,10.276661,4.833306,4.004050,1.703093,2.646029,3.963651,1.565274,1.603777,7.182543,...,-0.027461,True,False,False,True,False,True,True,True,0.0
2023-01-01 04:30:00,11.762656,10.215582,4.807457,3.978193,1.696518,2.639386,3.931292,1.552419,1.590777,7.137375,...,-0.026977,True,False,False,True,False,True,True,True,0.0
2023-01-01 04:45:00,11.717979,10.177689,4.798751,3.965234,1.696454,2.643612,3.909732,1.543855,1.582115,7.115800,...,-0.026783,True,False,False,True,False,True,True,True,0.0


## Save data

In [26]:
h5_fn = os.path.join(data_fn, 'drifted_data_y_2023_more_cos_phi.h5')
print(h5_fn)
simulation_data.to_hdf(h5_fn, mode='w', key='df')

/Users/torge/Development/master/masterthesis_code/02_Experimente/DataGenerator/data_modelling/drifted_data_y_2023_more_cos_phi.h5


In [27]:
cols = [s for s in simulation_data.columns if "loading" in s]
cols.append("drift_labels")
print(cols)

['line_1_1_loading', 'line_2_3_loading', 'line_3_4_loading', 'line_4_5_loading', 'line_5_6_loading', 'line_7_8_loading', 'line_8_9_loading', 'line_9_10_loading', 'line_10_11_loading', 'line_3_8_loading', 'line_12_13_loading', 'line_13_14_loading', 'line_6_7_loading', 'line_11_14_loading', 'line_14_8_loading', 'trafo_0_loading', 'trafo_1_loading', 'drift_labels']


In [28]:
simulation_data_reduced = simulation_data.copy()
simulation_data_reduced = simulation_data_reduced[cols]

In [29]:
h5_fn = os.path.join(data_fn, 'drifted_data_y_2023_reduced_more_cos_phi.h5')
print(h5_fn)
simulation_data_reduced.to_hdf(h5_fn, mode='w', key='df')

/Users/torge/Development/master/masterthesis_code/02_Experimente/DataGenerator/data_modelling/drifted_data_y_2023_reduced_more_cos_phi.h5
